In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -U tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
!pip install -q "tqdm>=4.36.1"

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import zipfile
import pandas as pd
import numpy as np
import pickle
from sklearn.utils import class_weight
from keras import regularizers
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from tensorflow.keras import models
from google.colab import drive
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import tensorflow_datasets as tfds
import tqdm
import tensorflow_addons as tfa
from tensorflow.keras.utils import image_dataset_from_directory
from PIL import Image
import os
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import shutil
import matplotlib.pyplot as plt
import cv2 as cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.applications import inception_v3

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!unzip -q "/content/drive/Shareddrives/DeepLearningProject/dataset_Pretrained_200/trainingSet.zip" -d "/content"

In [ ]:
!unzip -q "/content/drive/Shareddrives/DeepLearningProject/dataset_Pretrained_200/validationSet.zip" -d "/content"

In [ ]:
!unzip -q "/content/drive/Shareddrives/DeepLearningProject/testSet.zip" -d "/content"

In [ ]:
!rm -r "/content/trainingSet/"
!rm -r "/content/testSet/"
!rm -r "/content/validationSet/"

rm: cannot remove '/content/testSet/': No such file or directory


# Creation training validation and test sets

In [ ]:
BATCH_SIZE = 100
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

In [ ]:
training_dir = PATH = '/content/trainingSet/'
validation_dir = PATH = '/content/validationSet/'

training_dataset = tf.keras.utils.image_dataset_from_directory(
  training_dir,
  label_mode="categorical",
  labels="inferred",
  image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
  batch_size=BATCH_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
  validation_dir,
  label_mode="categorical",
  labels="inferred",
  image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
  batch_size=BATCH_SIZE)


Found 1000 files belonging to 5 classes.
Found 3510 files belonging to 5 classes.


In [ ]:
test_dir = PATH = '/content/testSet/'

test_dataset = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  label_mode="categorical",
  labels="inferred",
  image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
  batch_size=BATCH_SIZE)

In [ ]:
data_augmentation = keras.Sequential(
  [
  layers.RandomFlip("vertical"),
  layers.RandomFlip("horizontal"),
  layers.RandomZoom(0.1, fill_mode="constant", fill_value=0.0),
  layers.RandomRotation(factor=1, fill_mode="constant", fill_value=0.0),
  ]
)

# Define conv_base

In [ ]:
conv_base = keras.applications.inception_v3.InceptionV3(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
conv_base.trainable = False

### Exp 1


In [ ]:

inputs = keras.Input(shape=(224,224,3))
inputs = data_augmentation(inputs)
x = keras.applications.inception_v3.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs, outputs)
history = run_model(model, 0.0001, Adam)

### Exp 2

In [ ]:
inputs = keras.Input(shape=(224,224,3))
inputs = data_augmentation(inputs)
x = keras.applications.inception_v3.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs, outputs)
history = run_model(model, 0.0001, Adam)

### Exp 3

In [ ]:
inputs = keras.Input(shape=(224,224,3))
inputs = data_augmentation(inputs)
x = keras.applications.inception_v3.preprocess_input(inputs)
x = conv_base(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs, outputs)
history = run_model(model, 0.0001, Adam)

Training:   0%|           0/250 ETA: ?s,  ?epochs/s

Epoch 1/250


0/10           ETA: ?s - 

Epoch 1/250
10/10 [==============================] - ETA: 0s - loss: 0.7106 - accuracy: 0.1950
Epoch 00001: val_loss improved from inf to 0.53461, saving model to /content/models/weights.01-0.53.hdf5
10/10 [==============================] - 44s 4s/step - loss: 0.7106 - accuracy: 0.1950 - val_loss: 0.5346 - val_accuracy: 0.2709
Epoch 2/250


0/10           ETA: ?s - 

...

Epoch 25/250
10/10 [==============================] - ETA: 0s - loss: 0.3972 - accuracy: 0.5160
Epoch 00025: val_loss did not improve from 0.42737
10/10 [==============================] - 43s 4s/step - loss: 0.3972 - accuracy: 0.5160 - val_loss: 0.4761 - val_accuracy: 0.3490
Epoch 26/250


0/10           ETA: ?s - 

Epoch 26/250
10/10 [==============================] - ETA: 0s - loss: 0.3997 - accuracy: 0.5150
Epoch 00026: val_loss did not improve from 0.42737
10/10 [==============================] - 43s 4s/step - loss: 0.3997 - accuracy: 0.5150 - val_loss: 0.4910 - val_accuracy: 0.2960
Epoch 27/250


0/10           ETA: ?s - 

Epoch 27/250
10/10 [==============================] - ETA: 0s - loss: 0.3987 - accuracy: 0.5200
Epoch 00027: val_loss did not improve from 0.42737
10/10 [==============================] - 43s 4s/step - loss: 0.3987 - accuracy: 0.5200 - val_loss: 0.4560 - val_accuracy: 0.4305
Epoch 00027: early stopping


# Run Model

In [ ]:
def run_model(model, learning_rate, opt):
  
  global training_dataset
  global validation_dataset

  training_dataset = training_dataset.shuffle(3190, reshuffle_each_iteration=True)
  validation_dataset = validation_dataset.shuffle(3190, reshuffle_each_iteration=True)

  if opt=="Adam":
    optimizer = tf.keras.optimizers.Adam(learning_rate)
  elif opt=="SGD":
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
  else: 
    return "No optimizer"

  model.compile(run_eagerly=True,
                optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  checkpoint_filepath = '/content/models/weights.{epoch:02d}-{val_loss:.2f}.hdf5'

  checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, 
                              monitor='val_loss', 
                              verbose=1, 
                              save_best_only=True,
                              #save_freq = "epoch", 
                              mode='min')

  es = EarlyStopping(monitor='val_loss', 
                    mode='min', 
                    verbose=1, 
                    patience=10)

  tqdm_callback = tfa.callbacks.TQDMProgressBar()

  history = model.fit(training_dataset,
            validation_data=validation_dataset,
            callbacks=[es,checkpoint,tqdm_callback],
            epochs=250)
  return history



## Save model

In [ ]:
# save model and model history
model.save("/content/drive/Shareddrives/DeepLearningProject/models/InceptionV3/pretrained_InceptionV3_model1_200c.h5")

with open("/content/drive/Shareddrives/DeepLearningProject/histories/InceptionV3/pretrained_InceptionV3_history1_200c", 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

# Test model

In [ ]:
with open('/content/drive/Shareddrives/DeepLearningProject/histories/InceptionV3/pretrained_InceptionV3_history3_200c', 'rb') as file:
    history=pickle.load(file)

model = tf.keras.models.load_model("/content/drive/Shareddrives/DeepLearningProject/models/InceptionV3/pretrained_InceptionV3_model3_200c.h5")

In [ ]:
import matplotlib.pyplot as plt

print(history.keys())
# see how the accuracy and the loss change during training
plt.title('ACCURACY')
plt.plot(history['accuracy'],'bo-',label = 'Training Accuracy')
plt.plot(history['val_accuracy'],'ro-',label = 'Validation Accuracy')
plt.legend()
plt.show()
plt.title('LOSS')
plt.plot(history['loss'],'bo-',label = 'Training Loss')
plt.plot(history['val_loss'],'ro-',label = 'Validation Loss')
plt.legend()
plt.show()

AttributeError: ignored

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred_Y = model.predict(test_dataset, batch_size = 512, verbose = True)
pred_Y_cat = np.argmax(pred_Y, -1)
y_true = tf.concat([labels_batch for data_batch, labels_batch in test_dataset], axis = 0)
y_true_cat = np.argmax(y_true, -1)
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(y_true_cat, pred_Y_cat)))
print(classification_report(y_true_cat, pred_Y_cat))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(y_true_cat, pred_Y_cat), 
            annot=True, fmt="d", cbar = False, cmap = plt.cm.Blues)